In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

In [3]:
# Using ffill we optimize the code. It will fill all missing values with the previous non-nan value
data = pd.read_csv("data/kaggle_ner/ner_dataset.csv",
                   encoding="latin1").fillna(method="ffill")

data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [4]:
# !pip install editdistance

In [5]:
from itertools import chain
import editdistance

In [6]:
X_txt = list(data.groupby("Sentence #")['Word'].apply(list))

In [7]:
def edit_ditance_word(mistake, X_train_txt):
    # mistake = "Barchelona" 
    corpus = [w for seq in X_train_txt for w in seq]
    distances = [editdistance.eval(mistake, word) for word in corpus]
    return corpus[np.argmin(distances)], min(distances)

In [8]:
edit_ditance_word("Barchelona", X_txt)

('Barcelona', 1)

In [9]:
class BK_Tree():    
    def __init__(self, documents):
        self.root = Node(documents[0][0])
        for seq in documents:
            for word in seq:
                self.root.append(word)
        
    def append(self, word):
        self.root.append(word)
        
    def is_in_corpus(self, new_word):
        min_dist = float("+inf")
        min_word = ""
        return self.root.is_in_corpus(new_word, min_dist, min_word)
        
        
class Node():    
    def __init__(self, word):
        self.word = word
        self.dict_dist = {}
    
    def is_in_corpus(self, new_word, min_dist, min_word):
        if self.word == new_word:
            return (True, 0, self.word)
        dist = editdistance.eval(self.word, new_word)
        if dist<min_dist:
            min_dist = dist
            min_word = self.word
        if dist not in self.dict_dist:
            return (False, min_dist, min_word)
        return self.dict_dist[dist].is_in_corpus(new_word, min_dist, min_word)
    
    def append(self, new_word):
        dist = editdistance.eval(self.word, new_word)
        if dist not in self.dict_dist:
            if self.word!=new_word:
                self.dict_dist[dist] = Node(new_word)
        else:
            self.dict_dist[dist].append(new_word)    
            
            
bk_tree = BK_Tree(X_txt)


((True, 0, 'Barcelona'), (False, 9, 'surveillance'))

In [10]:
bk_tree.is_in_corpus("Barcelona"), bk_tree.is_in_corpus("Barcielona")

((True, 0, 'Barcelona'), (False, 6, 'discipline'))

In [ ]:
type(bk_tree)